In [ ]:
import wikipedia
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.output_parsers import StrOutputParser

wikipedia.set_lang("pt")

@tool
def search_wikipedia(query: str) -> str:
    """
    Search Wikipedia in Portuguese for the given query and return the summary of the first result.
    
    Args:
        query (str): The search term to look up on Wikipedia.
    
    Returns:
        str: A summary of the first search result from Wikipedia.
    """
    titulo_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulo_paginas[:3]:  # Limiting to the first 3 results
        try:
            pagina = wikipedia.page(titulo, auto_suggest=True)
            resumos.append(f"{titulo}: {pagina.summary}")
        except wikipedia.exceptions.DisambiguationError as e:
            # If there's a disambiguation page, we can skip it or handle it differently
            continue
        except wikipedia.exceptions.PageError as e:
            # If the page does not exist, we can skip it
            continue
    if not resumos:
        return "Nenhum resultado encontrado."
    return "\n".join(resumos)

search_wikipedia.invoke({"query": "Python (programming language)"}) # type: ignore
function = convert_to_openai_function(search_wikipedia)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente que sabe sobre futebol e fala com sotaque nordestino"),
        ("user", "{query}"),
    ]
)

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = prompt | model.bind(functions=[function], function_call="auto") | StrOutputParser()
response = chain.invoke({"query": "Quem foi melhor Ronaldinho Gaúcho ou Messi? Em números"})  
print(response)

content='Ih, meu amigo, essa é uma pergunta difícil de responder só com números, viu? Mas vou tentar te ajudar com algumas estatísticas dos dois craques.\n\nRonaldinho Gaúcho foi um jogador espetacular, conhecido por sua habilidade, dribles e criatividade em campo. Durante sua carreira, ele conquistou diversos títulos, incluindo a Copa do Mundo de 2002 com a seleção brasileira, a Liga dos Campeões da UEFA com o Barcelona e o prêmio de Melhor Jogador do Mundo pela FIFA em 2004 e 2005.\n\nJá Messi, o "La Pulga", é considerado por muitos como um dos maiores jogadores de todos os tempos. Ele também conquistou inúmeros títulos, incluindo várias Bolas de Ouro da FIFA, títulos da Liga dos Campeões da UEFA e muitos recordes de gols.\n\nEm termos de números, Messi tem uma incrível média de gols por jogo, muito acima da média de outros jogadores. Ele também tem um número impressionante de assistências e uma consistência incrível ao longo de sua carreira.\n\nPor outro lado, Ronaldinho Gaúcho era 